In [1]:
using Pkg
Pkg.add(PackageSpec(name="JWAS", rev="master"))
#Pkg.update()
Pkg.add("DataFrames")
Pkg.add("CSV")
using JWAS,JWAS.Datasets,DataFrames,CSV


    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
    Updating registry at `C:\Users\fmkay\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  [c9a035f4] ~ JWAS v1.1.2 ⇒ v1.2.1 `https://github.com/reworkhow/JWAS.jl.git#master`
    Updating `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
  [c9a035f4] ~ JWAS v1.1.2 ⇒ v1.2.1 `https://github.com/reworkhow/JWAS.jl.git#master`
Precompiling project...
  ✓ JWAS
  1 dependency successfully precompiled in 16 seconds (66 already precompiled)
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`


In [2]:
phenotypes = CSV.read("correlate_all_snps_haplos.csv",DataFrame,delim = ',',header=true,missingstrings=["NA"])
phenotypes = dropmissing(phenotypes, :qNurlogCort)
first(phenotypes,5)

,ID,PigID,Batch,BatchNum,Company,SowID,NurEnrich,EntryAge,cNurEntryAge
,Int64,Int64,Int64?,String?,String?,String?,Bool?,Int64?,Int64?
1,4935,3386,81,81,E,E8B260,missing,22,missing
2,4936,3387,81,81,E,W3W953,missing,22,missing
3,4937,3388,81,81,E,E8B205,missing,17,missing
4,4938,3389,81,81,E,E8B411,missing,19,missing
5,4939,3390,81,81,E,3240,missing,16,missing


In [3]:
Hair_G3=[7.5776  0
 0  0.289334E-01]
myPi=Dict([1.0; 1.0]=>0.000001,[1.0; 0.0]=>0.000999,[0.0; 1.0]=>0.000999,[0.0; 0.0]=>0.998001)
genotypes=get_genotypes("C:/Users/fmkay/Desktop/Fazhir/Hair_project/GWAS/dummy_genotypes.txt",header=true, Hair_G3, separator = ',', method="BayesB", quality_control=false, Pi = myPi, estimatePi = false);


The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095


In [4]:
Hair_model_equation = "qNurlogCort = Batch + EntryAge + SowID + qNurPenBatch + genotypes
Vocalization_number = Batch + EntryAge + SowID + qNurPenBatch + genotypes";

In [5]:
Hair_R= [1.4413 -0.284883
    -0.284883 0.733898E-01]
Hair_model = build_model(Hair_model_equation, Hair_R);

In [6]:
set_covariate(Hair_model, "EntryAge");

Hair_G1= [0.521 -0.284883
    -0.284883 0.733];
set_random(Hair_model,"qNurPenBatch",Hair_G1);
Hair_G2= [16.4413 0.03
    0.03 0.733898E-01];
set_random(Hair_model,"SowID",Hair_G2);


In [7]:
out1=runMCMC(Hair_model,phenotypes, missing_phenotypes=true, estimateScale=true, chain_length=1000,burnin = 0,output_heritability=true,  output_samples_frequency=10, output_samples_for_all_parameters = true);


running MCMC ... 100%|███████████████████████████████████| Time: 0:01:269mm9mm9m


The folder results is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Missing values are found in column Batch for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column EntryAge for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column SowID for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing 

In [16]:
cd("C:/Users/fmkay/Desktop/Fazhir/Hair_project/GWAS/results")
out = GWAS(Hair_model,"dummy_mapfile.txt","MCMC_samples_marker_effects_genotypes_qNurlogCort.txt","MCMC_samples_marker_effects_genotypes_Vocalization_number.txt",header=true,GWAS=true,threshold = 0.001,window_size="0.25 Mb",genetic_correlation=true,output_winVarProps = true)

running GWAS... 100%|████████████████████████████████████| Time: 0:00:17
running GWAS... 100%|████████████████████████████████████| Time: 0:00:13
calculating genomic correlation... 100%|█████████████████| Time: 0:00:14


Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance.


((87×13 DataFrame
 Row │ trait  window  chr     wStart    wEnd      start_SNP  end_SNP   numSNP  ⋯
     │ Int64  Int64   String  Int64     Int64     Int64      Int64     Int64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │     1      73  1       18000000  18250000   18007380  18245831      53  ⋯
   2 │     1      74  1       18250000  18500000   18256738  18445474      18
   3 │     1      62  1       15250000  15500000   15280579  15498706      22
   4 │     1      70  1       17250000  17500000   17252368  17498262      46
   5 │     1      87  1       21500000  21750000   21502842  21736247      59  ⋯
   6 │     1      86  1       21250000  21500000   21263684  21499391      52
   7 │     1      22  1        5250000   5500000    5252556   5496049      64
   8 │     1       5  1        1000000   1250000    1005658   1248474      81
   9 │     1      15  1        3500000   3750000    3501012   3743609      62  ⋯
  10 │     1      12  1     

In [ ]:
?runMCMC

In [ ]:
?GWAS
